In [ ]:
# imports
import torch
from DataObjects.DataLoader import DataLoader
from Architectures.Statistical import GMM

from pathlib import Path
import numpy as np

In [ ]:
# SETTING SEED
SEED = 42069    
torch.manual_seed(SEED)
np.random.seed(0)

In [ ]:
# set device to cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
DATA_PATH = Path("Data")

In [ ]:
# prepare Data Loader files
train_dir = DATA_PATH / Path("FFT_transformed", "training")
val_dir = DATA_PATH / Path("FFT_transformed", "validation")
test_dir = DATA_PATH / Path("FFT_transformed", "testing")

batch_size = 64

train_loader = DataLoader(
    data_dir=train_dir,
    data_type='fft',
    batch_size=batch_size,
    shuffle=True
)
val_loader = DataLoader(
    data_dir=val_dir,
    data_type='fft',
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    data_dir=test_dir,
    data_type='fft',
    batch_size=batch_size,
    shuffle=True
)

# Inspect class mapping and dataset size
print("Classes:", train_loader.class_to_idx)
print("# training batches:", len(train_loader))

print("# validation batches:", len(val_loader))

print("# testing batches:", len(test_loader))

In [ ]:
gmm_model = GMM(
    n_components=4,
    covariance_type='full',
    max_iter=100,
    random_state=42
)

In [ ]:
gmm_model.train(train_loader)

In [ ]:
results = gmm_model.evaluate(test_loader)
print("Test summary:")
for metric, value in results['summary'].items():
    print(f"  {metric}: {value:.4f}")